## Deliverable 2: Create a Customer Travel Destinations Map

In [146]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
import os

# Add parent folder to Python sys.path in order to import the config module
parent_folder = os.path.dirname(os.getcwd())
sys.path.append(parent_folder)

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [147]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Laguna,US,38.4210,-121.4238,60.64,66,1,8.05,clear sky
1,Puerto Ayora,EC,-0.7393,-90.3518,69.33,92,100,8.95,overcast clouds
2,Georgetown,MY,5.4112,100.3354,84.13,83,40,1.01,thunderstorm with light rain
3,Constitucion,CL,-35.3333,-72.4167,56.75,45,0,1.66,clear sky
4,Nome,US,64.5011,-165.4064,30.27,81,90,4.61,overcast clouds


In [148]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [149]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Laguna,US,38.4210,-121.4238,60.64,66,1,8.05,clear sky
1,Puerto Ayora,EC,-0.7393,-90.3518,69.33,92,100,8.95,overcast clouds
6,Komsomolskiy,UZ,40.4272,71.7189,77.72,36,43,6.91,scattered clouds
13,Kapaa,US,22.0752,-159.3190,75.52,81,75,1.99,moderate rain
16,Ribeira Grande,PT,38.5167,-28.7000,75.60,73,40,13.80,scattered clouds
17,Saint-Philippe,RE,-21.3585,55.7679,78.22,75,33,14.88,scattered clouds
18,Lagoa,PT,39.0500,-27.9833,68.56,82,100,18.30,overcast clouds
20,Busselton,AU,-33.6500,115.3333,60.04,80,100,7.81,overcast clouds
22,Emerald,AU,-23.5333,148.1667,69.35,70,24,19.84,few clouds
24,Albany,US,42.6001,-73.9662,64.99,88,95,4.23,overcast clouds


In [150]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [151]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [152]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Laguna,US,60.64,clear sky,38.4210,-121.4238,
1,Puerto Ayora,EC,69.33,overcast clouds,-0.7393,-90.3518,
6,Komsomolskiy,UZ,77.72,scattered clouds,40.4272,71.7189,
13,Kapaa,US,75.52,moderate rain,22.0752,-159.3190,
16,Ribeira Grande,PT,75.60,scattered clouds,38.5167,-28.7000,
17,Saint-Philippe,RE,78.22,scattered clouds,-21.3585,55.7679,
18,Lagoa,PT,68.56,overcast clouds,39.0500,-27.9833,
20,Busselton,AU,60.04,overcast clouds,-33.6500,115.3333,
22,Emerald,AU,69.35,few clouds,-23.5333,148.1667,
24,Albany,US,64.99,overcast clouds,42.6001,-73.9662,


In [153]:
import json

In [154]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # add location to params
    params["location"] = f"{lat},{lng}"
    # print(params)
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotel_json = requests.get(base_url, params).json()
    
    # print(json.dumps(hotel_json["results"][0], indent=4))
    # print(hotel_json["results"][0]["name"])
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_json["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        continue 

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [155]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df = clean_hotel_df.loc[clean_hotel_df["Hotel Name"] != ""]

In [156]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.set_index("City", inplace=True)
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [157]:
# reset index
clean_hotel_df.reset_index(inplace=True)

In [158]:
clean_hotel_df.columns

Index(['City', 'Country', 'Max Temp', 'Current Description', 'Lat', 'Lng',
       'Hotel Name'],
      dtype='object')

In [159]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [160]:
def create_heat_map(locations, weights, marker_info=None):
    """
    create new heat map with gmaps for locations and weights passed in
    
    args:
        locations: DataFrame with two columns [["Lat", "Lng"]]
        weights: List of max temperatures or humidity, or another weather metric
    """ 
    # create figure
    fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
    
    heat_layer = gmaps.heatmap_layer(locations, weights, dissipating=False, max_intensity=300, point_radius=4)
        
    # add the heatmap layers
    fig.add_layer(heat_layer)

    # add marker layer if marker_info argument is provided
    if (marker_info != None):
        marker_layer = gmaps.marker_layer(locations, info_box_content=marker_info)
        fig.add_layer(marker_layer)

    
    # call figure to plot the data
    return fig

In [161]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = create_heat_map(locations, max_temp, hotel_info)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))